In [1]:
import rasterio
import numpy as np
import matplotlib as mpl
import subprocess
from matplotlib import pyplot
import geopandas as gpd
import os, math
from matplotlib import pyplot as plt
from utils.landsatHepers import *
from shapely.wkt import dump as DumpPolygon, load as LoadPolygon

vn_reservoirs_path = 'vnreservoirs/VN_Reservoirs.shp'
df = gpd.read_file(vn_reservoirs_path)
plt.rcParams['figure.figsize'] = 7, 7
ho_tri_an = df[df.AREA_SKM == 277.4]

import json

In [2]:
def removeCloudFromQABand(rawData, qaData):
    returnValue = np.asarray(rawData)
    returnValue[np.asarray(qaData) != 0] = -1
    return returnValue

In [ ]:
for root, dir, filenames in os.walk('Landsat8/'):
    break

c = 0
for folder in dir:
    geotiffPath = os.path.join('Landsat8', folder)
    c += 1
    print('[{1}/{2}] Working on {0}... '.format(geotiffPath, c, len(dir)), end='')

    try:
        NDWI = rasterio.open(os.path.join(geotiffPath, 'TriAn_' + folder.split('_')[3] + '.TIF'))
                
        ndwi = NDWI.read(1)
        original_transform = NDWI.transform
        
        qaBand = rasterio.open(os.path.join(geotiffPath, 'TriAn_' + folder.split('_')[3] + '_BQA.TIF'))
#         ndwi = removeCloudFromQABand(ndwi, qaBand.read(1))     
        
        area, water_body = getWaterBody(ndwi, 'NDWI')

        if (area > 0):
            water_body = normalizePixelOnBoundaries(water_body)
            water_body_boundaries = find_boundaries(water_body, mode='outer', background=100).astype(np.int16)

            __shape, lShape = getMostSimilarShape(water_body_boundaries, original_transform, 
                                                  ho_tri_an.to_crs(NDWI.crs))

            new_shape_in_original_coordinates = [transform_geom(NDWI.crs, df.crs, mapping(__shape))]
            new_shape = shape(new_shape_in_original_coordinates[0])

            from shapely.wkt import dump as DumpPolygon, load as LoadPolygon

            wkt_path = os.path.join(geotiffPath, 'TriAn_' + folder.split('_')[3] + '_shape.wkt')
            DumpPolygon(new_shape, open(wkt_path, 'w'))
        
        _area = area * 900 / (1e6)
        data = json.dumps({
            'date': folder,
            'area': str(_area)
        })
        print(' --> OK. Area: {0}'.format(_area))
    except Exception as e:
        print(' --> ERROR: ' + str(e))

[1/182] Working on Landsat8/LC08_L1GT_124052_20130722_20170503_01_T2...  --> OK. Area: 74.0439
[2/182] Working on Landsat8/LC08_L1GT_124052_20131127_20170428_01_T2...  --> OK. Area: 0.9306
[3/182] Working on Landsat8/LC08_L1GT_124052_20150712_20170407_01_T2...  --> OK. Area: 0.3303
[4/182] Working on Landsat8/LC08_L1GT_124052_20150914_20170404_01_T2...  --> OK. Area: 74.7477
[5/182] Working on Landsat8/LC08_L1GT_124052_20160628_20170323_01_T2...  --> OK. Area: 0.0
[6/182] Working on Landsat8/LC08_L1TP_124052_20141130_20170417_01_T1...  --> OK. Area: 0.0
[7/182] Working on Landsat8/LC08_L1TP_124052_20141216_20170416_01_T1...  --> OK. Area: 243.0018
[8/182] Working on Landsat8/LC08_L1TP_124052_20150101_20170415_01_T1...  --> OK. Area: 77.499
[9/182] Working on Landsat8/LC08_L1TP_124052_20180109_20180119_01_T1... 